<a href="https://colab.research.google.com/github/Edenshmuel/PapaJohns_Data_Science_Project/blob/main/DeepAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.flush_and_unmount()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/cleaned_data.csv')

<ipython-input-2-431e256c197d>:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/cleaned_data.csv')


In [3]:
desc_encoding_map = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/desc_encoding_map.csv')

In [33]:
# !pip install pip==23.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [34]:
# !pip install gluonts==0.10.7 mxnet==1.8.0.post0

  Obtaining dependency information for gluonts==0.10.7 from https://files.pythonhosted.org/packages/b8/a9/b0ad9d1d0df70e269b68a4fa373b59e379054d6134146c494e252f1637a1/gluonts-0.10.7-py3-none-any.whl.metadata
  Obtaining dependency information for pandas~=1.0 from https://files.pythonhosted.org/packages/56/73/3351beeb807dca69fcc3c4966bcccc51552bd01549a9b13c04ab00a43f21/pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: gluonts
    Found existing installation: gluonts 0.13.3
    Uninstalling gluonts-0.13.3:
      Successfully uninstalled gluonts-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the pack

In [9]:
# !pip install numpy==1.23

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 25.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.23.0-cp311-cp311-linux_x86_64.whl size=19730355 sha256=8e0381b530c6d219f09b576cf7ceafe2b764a43bf8233119e0fabd1c351c9143
  Stored in directory: /root/.cache/pip/wheels/6d/36/1a/3ec6b85008bea3151efb003f5d41baa7bf4966cb43c1c2470b
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas 1.5.3 requires numpy>=1.23.2; python_version >= "3.11", but you have numpy 1.23.0 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you

In [4]:
from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

/usr/local/lib/python3.11/dist-packages/mxnet/ndarray/sparse.py:60: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  import scipy.sparse as spsp


In [5]:
import pandas as pd
import numpy as np

# הסרת עמודות לא רלוונטיות (קטגוריות, סכום, הזמנות, שמות חגים)
columns_to_drop = [
    'אחר', 'מנה עיקרית', 'קינוח', 'רוטב', 'שתייה', 'תוספת',
    'christian_holiday_name', 'jewish_holiday_name',
    'order', 'סכום'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

# ודא שהתאריכים בפורמט נכון
df['Date'] = pd.to_datetime(df['Date'])

# סיכום ביקושים לפי פריט ותאריך (שימור כל השורות, אבל נוודא שאין כפולים)
daily_demand = df.groupby(['Date', 'clean_desc_encoded'])['כמות'].sum().reset_index()

# הוספת פיצ'רים תלויי-תאריך
feature_cols = [
    'Year', 'Month', 'Day', 'Day_Name', 'Is_Weekend',
    'Season', 'WeekOfYear', 'Month_sin', 'Month_cos',
    'Day_Name_sin', 'Day_Name_cos', 'is_start_of_month',
    'is_end_of_month', 'is_near_jewish_holiday', 'is_day_before_new_year']

date_features = df[['Date'] + feature_cols].drop_duplicates('Date').sort_values('Date')

# ממזג את הפיצ'רים לפי תאריך
full_df = daily_demand.merge(date_features, left_on='Date', right_on='Date', how='left')

# לוודא שהכול מסודר לפי תאריך
full_df = full_df.sort_values(['clean_desc_encoded', 'Date']).reset_index(drop=True)

In [6]:
from gluonts.dataset.common import ListDataset

# רשימת כל הפיצ'רים שאנחנו מעבירים למודל
dynamic_features = feature_cols  # כל מה שנשאר לנו מפיצ'רים תלויי-תאריך

series_list = []
prediction_length = 7

for item_id, group in full_df.groupby('clean_desc_encoded'):
    group = group.sort_values('Date')

    target = group['כמות'].tolist()

    # בטיחות – אם אין מספיק תצפיות, לא נאמן עליו
    if len(target) < prediction_length * 2:
        continue

    # בניית פיצ'רים דינאמיים – רשימה של N רשימות (פיצ'רים x תאריכים)
    dynamic_feat = group[dynamic_features].T.values.tolist()

    entry = {
        "start": str(group['Date'].iloc[0].date()),
        "target": target,
        "feat_dynamic_real": dynamic_feat,
        "item_id": item_id
    }

    series_list.append(entry)

# יצירת קלט ל־DeepAR
train_ds = ListDataset(
    data_iter=series_list,
    freq="D")

In [7]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer

# נגדיר את המודל
estimator = DeepAREstimator(
    prediction_length=7,  # כמה ימים קדימה רוצים לחזות
    freq="D",
    trainer=Trainer(epochs=10),
    use_feat_dynamic_real=True)

# אימון המודל על הנתונים
predictor = estimator.train(train_ds)

100%|██████████| 50/50 [00:03<00:00, 14.69it/s, epoch=10/10, avg_epoch_loss=2.1]


In [8]:
def create_features_for_date(date: pd.Timestamp) -> list:
    year = date.year
    month = date.month
    day = date.day
    day_name = date.weekday() + 1  # 1 = ראשון, 7 = שבת
    is_weekend = 1 if day_name in [6, 7] else 0
    is_start_of_month = 1 if day == 1 else 0
    is_end_of_month = 1 if (date + pd.Timedelta(days=1)).day == 1 else 0
    week_of_year = date.isocalendar()[1]
    day_name_sin = np.sin(2 * np.pi * day_name / 7)
    day_name_cos = np.cos(2 * np.pi * day_name / 7)
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    if month in [12, 1, 2]:
        season = 2
    elif month in [3, 4, 5]:
        season = 3
    elif month in [6, 7, 8]:
        season = 4
    else:
        season = 1
    jewish_holidays = pd.to_datetime([
        "2025-06-02", "2025-10-03", "2025-10-12"
    ])
    is_near_jewish_holiday = int(any(abs((date - h).days) <= 2 for h in jewish_holidays))
    is_day_before_new_year = int(date == pd.to_datetime(f"{year}-12-31"))
    return [
        year, month, day, day_name, is_weekend,
        season, week_of_year, month_sin, month_cos,
        day_name_sin, day_name_cos,
        is_start_of_month, is_end_of_month,
        is_near_jewish_holiday, is_day_before_new_year]

In [9]:
desc_encoding_map.columns = ['item_name', 'clean_desc_encoded']

In [10]:
desc_encoding_map

,item_name,clean_desc_encoded
0,ביאנקה 14,0
1,טוסקנית 14,1
2,המומלצת 14,2
3,טוסקנית בצק דק 14,3
4,היוונית דקה 14,4
...,...,...
251,2 פיצות משפחתיות ב89 שח,251
252,2 תוספות צמחוניות חבר,252
253,פיצה משפחתית l מנה נלוות,253
254,פיצה אישית פחית,254


In [18]:
from datetime import timedelta
from gluonts.dataset.common import ListDataset
import numpy as np
import pandas as pd

# קלט התאריך לחיזוי מהמשתמש (בפורמט YYYY-MM-DD)
input_date = input("📅 הזן תאריך לחיזוי (בפורמט YYYY-MM-DD): ")
input_date = pd.to_datetime(input_date)

results = []

# עבור כל פריט ברשימת הסיריות
for item in series_list:
    item_id = item["item_id"]
    start_date = item["start"].to_timestamp()
    target_length = len(item["target"])  # אורך הסדרה ההיסטורית
    last_date = start_date + timedelta(days=target_length - 1)

    # העתקת פיצ'רים היסטוריים (על פי ציפיות - 15 פיצ'רים)
    original_feat = np.array(item["feat_dynamic_real"])

    # בדיקה: מספר השורות (הפיצ'רים) צריך להיות 15
    if original_feat.shape[0] != 15:
        print(f"⚠️ פריט {item_id} עם פיצ'רים לא תקינים: {original_feat.shape}")
        continue  # אם לא תקין – מעבר לפריט הבא

    # חישוב מספר הימים בין התאריך האחרון בהיסטוריה לבין התאריך שהוזן
    days_needed = max(0, (input_date - last_date).days + 1)

    # הגדרת אורך התחזית:
    # אם התאריך שהוזן נמצא בתוך טווח התחזית (המינימלי הוא prediction_length),
    # נבחר את האורך הגדול יותר.
    forecast_horizon = max(prediction_length, days_needed)

    # יצירת רשימת תאריכים לעתיד מהתאריך האחרון + 1 עד למספר הימים הנדרשים (forecast_horizon)
    future_dates = [last_date + timedelta(days=i + 1) for i in range(forecast_horizon)]
    # יצירת הפיצ'רים לעתיד עבור כל תאריך באמצעות הפונקציה create_features_for_date
    future_features = [create_features_for_date(d) for d in future_dates]

    # המרה למטריצה תוך טרנספוזיציה כדי לקבל צורה (15, forecast_horizon)
    future_dynamic_feat = np.array(future_features).T

    print(f"\n🔍 פריט: {item_id}")
    print("היסטוריה:", original_feat.shape)
    print("עתידי:   ", future_dynamic_feat.shape)

    # איחוד הפיצ'רים ההיסטוריים עם הפיצ'רים העתידיים לאורך ציר העמודות (axis=1)
    full_feat = np.concatenate([original_feat, future_dynamic_feat], axis=1)

    # בניית סט הנתונים לחיזוי בהתאם לדרישות GluonTS:
    # - "target": הסדרה ההיסטורית המקורית
    # - "feat_dynamic_real": המטריצה המאוחדת בגודל (15, len(target) + forecast_horizon)
    forecast_ds = ListDataset([{
        "start": item['start'],
        "target": item['target'],
        "feat_dynamic_real": full_feat,
        "item_id": item_id
    }], freq="D")

    # הפעלת המודל לביצוע התחזית
    forecast_it = predictor.predict(forecast_ds)
    forecast = next(iter(forecast_it))

    # יצירת רשימת תאריכים המתאימה לאורך התחזית האמיתית של המודל (אורך prediction_length)
    forecast_dates = pd.date_range(start=forecast.start_date.to_timestamp(), periods=prediction_length)

    # במידה והתאריך שהוזן נמצא בתוך טווח התחזית, נחשב את החיזוי עבורו
    if input_date in forecast_dates:
        idx = forecast_dates.get_loc(input_date)
        predicted_demand = forecast.mean[idx]
        # עיגול החיזוי כלפי מעלה והוספת התוצאה לרשימת התוצאות
        results.append((item_id, np.ceil(predicted_demand)))

# הדפסת תוצאות החיזוי
print("תוצאות החיזוי:")
for item_id, demand in results:
    print(f"פריט {item_id}: {demand}")

📅 הזן תאריך לחיזוי (בפורמט YYYY-MM-DD): 2025-01-03

🔍 פריט: 0
היסטוריה: (15, 359)
עתידי:    (15, 741)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 366 and the array at index 2 has size 1100

In [17]:
# from datetime import timedelta
# from gluonts.dataset.common import ListDataset
# import numpy as np
# import pandas as pd

# input_date = input("📅 הזן תאריך לחיזוי (בפורמט YYYY-MM-DD): ")
# input_date = pd.to_datetime(input_date)

# results = []

# for item in series_list:
#     item_id = item["item_id"]
#     start_date = item["start"].to_timestamp()
#     target_length = len(item["target"])
#     last_date = start_date + timedelta(days=target_length - 1)

#     original_feat = np.array(item["feat_dynamic_real"])  # עותק חדש

#     # 🔍 בדיקה: האם המבנה של הפיצ'רים תקין?
#     actual_feat_shape = np.array(item["feat_dynamic_real"]).shape
#     if actual_feat_shape[0] != 15:
#         print(f"⚠️ פריט {item_id} עם פיצ'רים לא תקינים: {actual_feat_shape}")
#         continue  # ממשיכים לפריט הבא

#     # כמה ימים צריך כדי להגיע לתאריך שהוזן
#     days_needed = (input_date - last_date).days + 1
#     days_to_extend = max(0, days_needed + prediction_length)

#     # יצירת הפיצ'רים העתידיים רק עד כמה שצריך
#     future_dates = [last_date + timedelta(days=i + 1) for i in range(days_to_extend)]
#     future_features = [create_features_for_date(d) for d in future_dates]
#     future_dynamic_feat = np.array(future_features).T.tolist()

#     print(f"\n🔍 פריט: {item_id}")
#     print("היסטוריה:", np.array(item["feat_dynamic_real"]).shape)
#     print("עתידי:   ", np.array(future_dynamic_feat).shape)

#     # full_feat = np.concatenate([item["feat_dynamic_real"], future_dynamic_feat], axis=1)
#     full_feat = np.concatenate([original_feat, future_dynamic_feat], axis=1)

#     forecast_ds = ListDataset([{
#         "start": item['start'],
#         "target": item['target'],
#         "feat_dynamic_real": full_feat,
#         "item_id": item_id
#     }], freq="D")

#     forecast_it = predictor.predict(forecast_ds)
#     forecast = next(iter(forecast_it))

#     forecast_dates = pd.date_range(start=forecast.start_date.to_timestamp(), periods=prediction_length)

#     if input_date in forecast_dates:
#         idx = forecast_dates.get_loc(input_date)
#         predicted_demand = forecast.mean[idx]
#         results.append((item_id, np.ceil(predicted_demand)))  # עיגול כלפי מעלה

📅 הזן תאריך לחיזוי (בפורמט YYYY-MM-DD): 2025-01-03

🔍 פריט: 0
היסטוריה: (15, 359)
עתידי:    (15, 748)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 366 and the array at index 2 has size 1107

In [ ]:
actual_feat_shape = np.array(item["feat_dynamic_real"]).shape
if actual_feat_shape[0] != 15:
    print(f"⚠️ פריט {item_id} עם פיצ'רים לא תקינים: {actual_feat_shape}")
    continue  # נדלג על פריטים עם מבנה שגוי

In [44]:
results_df = pd.DataFrame(results, columns=["clean_desc_encoded", "כמות חזויה"])
results_df = results_df.merge(desc_map, on="clean_desc_encoded", how="left")
results_df = results_df[["item_name", "כמות חזויה"]].rename(columns={"item_name": "פריט"})
results_df["כמות חזויה"] = results_df["כמות חזויה"].astype(int)
results_df = results_df.sort_values(by="כמות חזויה", ascending=False)

if results_df.empty:
    print("😕 לא נמצאה תחזית עבור התאריך הזה.")
else:
    print("📈 תחזית ביקוש לפי פריט:")
    print(results_df)


🔍 פריט: 0
• אורך הסדרה: 359
• תאריך אחרון בסדרה: 2022-12-25 00:00:00
• אורך feat_dynamic_real: (15, 359)
🔍 פריט: 1
• אורך הסדרה: 349
• תאריך אחרון בסדרה: 2022-12-15 00:00:00
• אורך feat_dynamic_real: (15, 349)
🔍 פריט: 2
• אורך הסדרה: 130
• תאריך אחרון בסדרה: 2022-05-10 00:00:00
• אורך feat_dynamic_real: (15, 130)
🔍 פריט: 3
• אורך הסדרה: 167
• תאריך אחרון בסדרה: 2022-06-16 00:00:00
• אורך feat_dynamic_real: (15, 167)
🔍 פריט: 4
• אורך הסדרה: 315
• תאריך אחרון בסדרה: 2022-11-11 00:00:00
• אורך feat_dynamic_real: (15, 315)
🔍 פריט: 5
• אורך הסדרה: 112
• תאריך אחרון בסדרה: 2022-04-22 00:00:00
• אורך feat_dynamic_real: (15, 112)
🔍 פריט: 6
• אורך הסדרה: 212
• תאריך אחרון בסדרה: 2022-07-31 00:00:00
• אורך feat_dynamic_real: (15, 212)
🔍 פריט: 7
• אורך הסדרה: 311
• תאריך אחרון בסדרה: 2022-11-07 00:00:00
• אורך feat_dynamic_real: (15, 311)
🔍 פריט: 8
• אורך הסדרה: 614
• תאריך אחרון בסדרה: 2023-09-06 00:00:00
• אורך feat_dynamic_real: (15, 614)
🔍 פריט: 9
• אורך הסדרה: 494
• תאריך אחרון בסדרה: 2023-0